## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Anadyr,RU,64.7500,177.4833,-4.00,77,20,13.42,few clouds
1,1,Yellowknife,CA,62.4560,-114.3525,9.00,61,20,9.22,few clouds
2,2,Charleston,US,32.7766,-79.9309,84.20,73,1,14.97,clear sky
3,3,Meulaboh,ID,4.1363,96.1285,76.23,84,100,1.39,overcast clouds
4,4,Rikitea,PF,-23.1203,-134.9692,79.23,71,1,18.88,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Charleston,US,32.7766,-79.9309,84.20,73,1,14.97,clear sky
3,3,Meulaboh,ID,4.1363,96.1285,76.23,84,100,1.39,overcast clouds
4,4,Rikitea,PF,-23.1203,-134.9692,79.23,71,1,18.88,clear sky
15,15,Cabo San Lucas,MX,22.8909,-109.9124,75.99,53,40,13.80,scattered clouds
20,20,Cidreira,BR,-30.1811,-50.2056,76.23,83,98,8.12,moderate rain
21,21,Dingle,PH,10.9995,122.6711,75.20,91,75,7.14,broken clouds
22,22,Souillac,MU,-20.5167,57.5167,77.00,94,75,3.44,broken clouds
27,27,Kavaratti,IN,10.5669,72.6420,82.90,74,25,3.91,scattered clouds
29,29,Hithadhoo,MV,-0.6000,73.0833,82.33,71,92,6.71,overcast clouds
31,31,Malindi,KE,-3.2175,40.1191,82.40,78,20,9.22,few clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                282
City                   282
Country                282
Lat                    282
Lng                    282
Max Temp               282
Humidity               282
Cloudiness             282
Wind Speed             282
Current Description    282
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Charleston,US,84.20,clear sky,32.7766,-79.9309,
3,Meulaboh,ID,76.23,overcast clouds,4.1363,96.1285,
4,Rikitea,PF,79.23,clear sky,-23.1203,-134.9692,
15,Cabo San Lucas,MX,75.99,scattered clouds,22.8909,-109.9124,
20,Cidreira,BR,76.23,moderate rain,-30.1811,-50.2056,
21,Dingle,PH,75.20,broken clouds,10.9995,122.6711,
22,Souillac,MU,77.00,broken clouds,-20.5167,57.5167,
27,Kavaratti,IN,82.90,scattered clouds,10.5669,72.6420,
29,Hithadhoo,MV,82.33,overcast clouds,-0.6000,73.0833,
31,Malindi,KE,82.40,few clouds,-3.2175,40.1191,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Charleston,US,84.20,clear sky,32.7766,-79.9309,The Mills House Wyndham Grand Hotel
3,Meulaboh,ID,76.23,overcast clouds,4.1363,96.1285,Kost Pria
4,Rikitea,PF,79.23,clear sky,-23.1203,-134.9692,Pension Maro'i
15,Cabo San Lucas,MX,75.99,scattered clouds,22.8909,-109.9124,Hotel Tesoro Los Cabos
20,Cidreira,BR,76.23,moderate rain,-30.1811,-50.2056,Hotel Castelo
21,Dingle,PH,75.20,broken clouds,10.9995,122.6711,SEGAYA PROPERTY
22,Souillac,MU,77.00,broken clouds,-20.5167,57.5167,Shanti Maurice Resort & Spa
27,Kavaratti,IN,82.90,scattered clouds,10.5669,72.6420,Hotels in Lakshadweep Islands
29,Hithadhoo,MV,82.33,overcast clouds,-0.6000,73.0833,Scoop Guest House
31,Malindi,KE,82.40,few clouds,-3.2175,40.1191,African House Resort


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]


# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [12]:
# 11a. Add a marker layer for each city to the map. 

info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))